Run these commands in a Terminator:
ros2 pkg list|grep turtle
ros2 run turtlesim turtlesim_node 
Split the screen vertically with: ctrl+shift+e
ros2 topic list
rqt_graph

In [1]:
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
from turtlesim.msg import Pose
import subprocess
import time
import threading
import ipywidgets as widgets
from IPython.display import display, clear_output

# Clean turtlesim workspace
def reset_turtlesim():
    """Reset turtlesim to clean state"""
    try:
        subprocess.run(['ros2', 'service', 'call', '/reset', 'std_srvs/srv/Empty'], 
                      capture_output=True, timeout=3)
        subprocess.run(['ros2', 'service', 'call', '/clear', 'std_srvs/srv/Empty'], 
                      capture_output=True, timeout=3)
        print("Turtlesim workspace cleaned successfully")
    except Exception as e:
        print(f"Could not clean workspace: {e}")

# Run cleanup
reset_turtlesim()

# Basic node setup for exercises
class TurtleController(Node):
    def __init__(self):
        super().__init__('turtle_workshop_controller')
        self.cmd_pub = self.create_publisher(Twist, '/turtle1/cmd_vel', 10)
        self.pose_sub = self.create_subscription(Pose, '/turtle1/pose', self.pose_callback, 10)
        self.current_pose = None
        
    def pose_callback(self, msg):
        self.current_pose = msg
        
    def send_command(self, linear, angular):
        msg = Twist()
        msg.linear.x = float(linear)
        msg.angular.z = float(angular)
        self.cmd_pub.publish(msg)

if not rclpy.ok():
    rclpy.init()

controller = TurtleController()

def spin_node():
    rclpy.spin(controller)

threading.Thread(target=spin_node, daemon=True).start()
print("Setup complete - workspace cleaned and controller ready")

Could not clean workspace: Command '['ros2', 'service', 'call', '/clear', 'std_srvs/srv/Empty']' timed out after 3 seconds
Setup complete - workspace cleaned and controller ready


[WARN] [1758141270.075343015] [rcl]: ROS_LOCALHOST_ONLY is deprecated but still honored if it is enabled. Use ROS_AUTOMATIC_DISCOVERY_RANGE and ROS_STATIC_PEERS instead.
[WARN] [1758141270.075369435] [rcl]: 'localhost_only' is disabled, 'automatic_discovery_range' and 'static_peers' will be used.


In [1]:
import os
import rclpy
from rclpy.node import Node

# Initialize ROS 2
rclpy.init()

print("✅ ROS 2 initialized successfully!")
print("ROS_DISTRO =", os.getenv('ROS_DISTRO', 'Not set'))

✅ ROS 2 initialized successfully!
ROS_DISTRO = jazzy


[WARN] [1758367591.788306768] [rcl]: ROS_LOCALHOST_ONLY is deprecated but still honored if it is enabled. Use ROS_AUTOMATIC_DISCOVERY_RANGE and ROS_STATIC_PEERS instead.
[WARN] [1758367591.788327007] [rcl]: 'localhost_only' is disabled, 'automatic_discovery_range' and 'static_peers' will be used.


In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String

class MinimalPublisher(Node):
    def __init__(self):
        super().__init__('minimal_publisher')
        self.publisher_ = self.create_publisher(String, 'topic', 10)
        timer_period = 0.5  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.i = 0

    def timer_callback(self):
        msg = String()
        msg.data = f'Hello from Jupyter! Count: {self.i}'
        self.publisher_.publish(msg)
        self.get_logger().info(f'Publishing: "{msg.data}"')
        self.i += 1
        if self.i > 5:  # Stop after 6 messages
            self.timer.cancel()
            self.get_logger().info("⏹️  Stopping publisher after 6 messages.")

# Create and run node
node = MinimalPublisher()
try:
    rclpy.spin(node)
except KeyboardInterrupt:
    pass
finally:
    node.destroy_node()
    # Don't shutdown rclpy yet — we'll use it in the next cell

[INFO] [1758367599.751058186] [minimal_publisher]: Publishing: "Hello from Jupyter! Count: 0"
[INFO] [1758367600.172436894] [minimal_publisher]: Publishing: "Hello from Jupyter! Count: 1"
[INFO] [1758367600.673324220] [minimal_publisher]: Publishing: "Hello from Jupyter! Count: 2"
[INFO] [1758367601.173057420] [minimal_publisher]: Publishing: "Hello from Jupyter! Count: 3"
[INFO] [1758367601.672876537] [minimal_publisher]: Publishing: "Hello from Jupyter! Count: 4"
[INFO] [1758367602.171475447] [minimal_publisher]: Publishing: "Hello from Jupyter! Count: 5"
[INFO] [1758367602.174362626] [minimal_publisher]: ⏹️  Stopping publisher after 6 messages.


In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String

class MinimalSubscriber(Node):
    def __init__(self):
        super().__init__('minimal_subscriber')
        self.subscription = self.create_subscription(
            String,
            'topic',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning
        self.msg_count = 0

    def listener_callback(self, msg):
        self.get_logger().info(f'🔍 Received: "{msg.data}"')
        self.msg_count += 1
        if self.msg_count >= 6:
            self.get_logger().info("✅ Received 6 messages. Stopping subscriber.")
            raise SystemExit  # Exit spin gracefully

# Create and run subscriber
sub_node = MinimalSubscriber()
try:
    rclpy.spin(sub_node)
except SystemExit:
    pass
finally:
    sub_node.destroy_node()

In [ ]:
import subprocess

def run_ros_command(cmd):
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return result.stdout.strip()

print("🔍 Active Nodes:")
print(run_ros_command("ros2 node list"))

print("\n📋 Topics:")
print(run_ros_command("ros2 topic list"))

print("\n📊 Topic Info:")
print(run_ros_command("ros2 topic info /topic"))

print("\n📬 Echoing live messages:")
print(run_ros_command("ros2 topic echo /topic --once"))

In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String

class MinimalPublisher(Node):
    def __init__(self):
        super().__init__('minimal_publisher')
        self.publisher_ = self.create_publisher(String, 'topic', 10)
        timer_period = 1.0
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.i = 0

    def timer_callback(self):
        msg = String()
        msg.data = f"Hello from Jupyter! Count: {self.i}"
        self.publisher_.publish(msg)
        self.get_logger().info(f'Publishing: "{msg.data}"')
        self.i += 1
        if self.i > 5:
            self.timer.cancel()

pub_node = MinimalPublisher()
try:
    rclpy.spin(pub_node)
except KeyboardInterrupt:
    pass
finally:
    pub_node.destroy_node()

In [ ]:
print(run_ros_command("ros2 topic echo /topic --once"))

In [ ]:
import os
os.system("ros2 node list")

In [ ]:
!ros2 node list

In [ ]:
import subprocess

def run_ros2(cmd):
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        return result.stdout.strip()
    else:
        return f"❌ Error: {result.stderr.strip()}"

# Example usage
print("🔍 Nodes:")
print(run_ros2("ros2 node list"))

print("\n📋 Topics:")
print(run_ros2("ros2 topic list"))

print("\n📊 Info on /topic:")
print(run_ros2("ros2 topic info /topic"))

print("\n📬 Echo one message:")
print(run_ros2("ros2 topic echo /topic --once"))

In [ ]:
import os

os.system("ros2 node list")
os.system("ros2 topic list")

In [ ]:
import os
import subprocess

# Source ROS 2 setup script
setup_script = "/opt/ros/humble/setup.bash"  # Change to your distro: foxy, iron, etc.

# Check if file exists
if os.path.exists(setup_script):
    print("✅ Found ROS 2 setup script:", setup_script)
    
    # Source it
    result = subprocess.run(f"source {setup_script} && echo 'ROS 2 sourced'", shell=True, capture_output=True, text=True)
    print(result.stdout.strip())
else:
    print("❌ Setup script not found:", setup_script)
    print("Check your ROS 2 installation.")

In [ ]:
import os

# Try running ros2 commands
os.system("ros2 --version")
os.system("ros2 node list")
os.system("ros2 topic list")

In [ ]:
def run_ros2(cmd):
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        return result.stdout.strip()
    else:
        return f"❌ Error: {result.stderr.strip()}"

print(run_ros2("ros2 node list"))
print(run_ros2("ros2 topic list"))

In [ ]:
import threading
import time
import rclpy
from rclpy.node import Node
from example_interfaces.srv import AddTwoInts

# ----------------------------
# SERVICE SERVER NODE
# ----------------------------
class AddTwoIntsServer(Node):
    def __init__(self):
        super().__init__('add_two_ints_server')
        self.srv = self.create_service(AddTwoInts, 'add_two_ints', self.add_two_ints_callback)
        self.get_logger().info('✅ AddTwoInts Server Ready!')

    def add_two_ints_callback(self, request, response):
        response.sum = request.a + request.b
        self.get_logger().info(f'📥 Received request: {request.a} + {request.b} = {response.sum}')
        return response

# ----------------------------
# SERVICE CLIENT NODE
# ----------------------------
class AddTwoIntsClient(Node):
    def __init__(self):
        super().__init__('add_two_ints_client')
        self.client = self.create_client(AddTwoInts, 'add_two_ints')
        while not self.client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('⏳ Waiting for server...')
        self.get_logger().info('🔗 Service server found!')

    def send_request(self, a, b):
        request = AddTwoInts.Request()
        request.a = a
        request.b = b
        future = self.client.call_async(request)
        rclpy.spin_until_future_complete(self, future)
        return future.result()

# ----------------------------
# INITIALIZE ROS 2 & RUN
# ----------------------------
def spin_node(node):
    rclpy.spin(node)

# Initialize ROS 2
rclpy.init()

# Create server and client
server_node = AddTwoIntsServer()
client_node = AddTwoIntsClient()

# Start server in background thread
server_thread = threading.Thread(target=spin_node, args=(server_node,), daemon=True)
server_thread.start()

# Wait a moment for server to be ready
time.sleep(1)

# Send requests
print("🧮 Sending service requests...")
try:
    for i in range(3):
        a, b = i+1, i+2
        result = client_node.send_request(a, b)
        print(f"🟢 {a} + {b} = {result.sum}")
except Exception as e:
    print(f"❌ Error: {e}")

# Cleanup
client_node.destroy_node()
server_node.destroy_node()
rclpy.shutdown()

print("✅ Service test completed!")


In [ ]:
import threading
import time
import rclpy
from rclpy.node import Node
from example_interfaces.srv import AddTwoInts

# ----------------------------
# SERVICE SERVER NODE
# ----------------------------
class AddTwoIntsServer(Node):
    def __init__(self):
        super().__init__('add_two_ints_server')
        self.srv = self.create_service(AddTwoInts, 'add_two_ints', self.add_two_ints_callback)
        self.get_logger().info('✅ AddTwoInts Server Started!')

    def add_two_ints_callback(self, request, response):
        response.sum = request.a + request.b
        self.get_logger().info(f'📥 Request: {request.a} + {request.b} = {response.sum}')
        return response

# ----------------------------
# SERVICE CLIENT NODE
# ----------------------------
class AddTwoIntsClient(Node):
    def __init__(self):
        super().__init__('add_two_ints_client')
        self.client = self.create_client(AddTwoInts, 'add_two_ints')
        while not self.client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('⏳ Waiting for server...')
        self.get_logger().info('🔗 Service server found!')

    def send_request(self, a, b):
        request = AddTwoInts.Request()
        request.a = a
        request.b = b
        future = self.client.call_async(request)
        rclpy.spin_until_future_complete(self, future)
        return future.result()

# ----------------------------
# INITIALIZE ROS 2 & RUN
# ----------------------------
def spin_node(node):
    rclpy.spin(node)

# Initialize ROS 2
rclpy.init()

# Create nodes
server_node = AddTwoIntsServer()
client_node = AddTwoIntsClient()

# Start server in background thread
server_thread = threading.Thread(target=spin_node, args=(server_node,), daemon=True)
server_thread.start()

# Wait for server to be ready
time.sleep(1)

# Send requests
print("🧮 Sending service requests...")
try:
    for i in range(3):
        a, b = i+1, i+2
        result = client_node.send_request(a, b)
        print(f"🟢 {a} + {b} = {result.sum}")
except Exception as e:
    print(f"❌ Error: {e}")

# Cleanup
client_node.destroy_node()
server_node.destroy_node()
rclpy.shutdown()

print("✅ Service test completed!")

In [ ]:
import os
import subprocess

# Try to run basic ROS 2 command
result = subprocess.run("ros2 --version", shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✅ ROS 2 is available:", result.stdout.strip())
else:
    print("❌ ROS 2 CLI not found. Error:", result.stderr.strip())
    print("\n⚠️  You must install and source ROS 2 Jazzy first.")

In [ ]:
try:
    import rclpy
    print("✅ rclpy imported successfully")
    print("rclpy version:", rclpy.__version__)
except Exception as e:
    print("❌ Failed to import rclpy:", e)
    print("\n💡 Fix: Source ROS 2 or install it.")